In [ ]:
import pupil_classv2 as pc
import math
from statistics import mean
import csv
from timestamps import new_timestamps
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import *
import tdt
from scipy.signal import find_peaks, peak_widths,resample,correlate, correlation_lags,peak_prominences,resample,butter, lfilter, freqz,medfilt
from scipy.stats import pearsonr,wilcoxon,mannwhitneyu,kruskal
import os
from fp_helper import *
from matplotlib import rcParams
import cv2
from scipy.stats import ttest_ind,ttest_rel,zscore
from sklearn.linear_model import Ridge,RidgeCV
from sklearn.model_selection import KFold,cross_val_score
from random import randint
from scipy.ndimage import median_filter
import scikit_posthocs as sp
from matplotlib.patches import ConnectionPatch

params = {
        "font.family" : "Arial",
        'pdf.fonttype' : 42,
        'axes.labelsize': 10,
        'axes.titlesize': 11,
        'axes.linewidth': 0.5,
        'xtick.labelsize':9,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'ytick.labelsize':9,
        'axes.spines.top':False,
        'axes.spines.right':False
         }
rcParams['figure.figsize'] = 21.7,8.27
rcParams.update(params)
def format_ax(ax,xlim,ylim,xspace,yspace):
   

    ax.set_xticks(np.arange(xlim[0],xlim[1]+ xspace,xspace))
    ax.set_yticks(np.arange(ylim[0],ylim[1]+ yspace,yspace))

    ax.set(ylim=(ylim[0], ylim[1]))
    ax.set(xlim=(xlim[0], xlim[1]))


In [ ]:
###helper function to get pupil without interpolating
def get_raw_pupil(id,date,sess = 1):
    csv_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_vid.mjpegDLC_resnet50_DLC_8points_pupilOct25shuffle1_2940000.csv"

    fps = 20
    thresh = .95
    drop_count = 0
    line_count = 0
    Areas = []

    with open(csv_path, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        line_count = 0
        for row in csvreader:
            if line_count >=3:
                new_pupil = pc.load_pupil(row)
                if (new_pupil.rpl > thresh) & (new_pupil.lpl > thresh) & (new_pupil.lvpl > thresh) & (new_pupil.rvpl > thresh) &( new_pupil.rdpl > thresh)&(new_pupil.ldpl > thresh) & (new_pupil.vpl > thresh) & (new_pupil.dpl > thresh):
                    area = new_pupil.get_area()
                    Areas.append(float(area))
                else:
                    print(1)
                    Areas.append(np.nan)
            line_count +=1
    return zscore(Areas,nan_policy = "omit")

In [ ]:
###helper function to get dff

def get_tone_dff(id,date):
    path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}"
    for filename in os.listdir(path):
        if os.path.isdir(os.path.join(path, filename)):
            blockpath = (os.path.join(path, filename))
    data = tdt.read_block(blockpath)
    GCAMP = '_465A' # GCaMP channel
    ISOS = '_405A' # Isosbestic channel
    TONE = 'PC2_'
    stamps = 'PC0_'
    time = np.linspace(1,len(data.streams[GCAMP].data), len(data.streams[GCAMP].data))/data.streams[GCAMP].fs
    N = 10 # Average every 10 samples into 1 value
    F405 = []
    F465 = []

    for i in range(0, len(data.streams[GCAMP].data), N):
        F465.append(np.mean(data.streams[GCAMP].data[i:i+N-1])) # This is the moving window mean
    data.streams[GCAMP].data = F465

    for i in range(0, len(data.streams[ISOS].data), N):
        F405.append(np.mean(data.streams[ISOS].data[i:i+N-1]))
    data.streams[ISOS].data = F405

    #decimate time array to match length of demodulated stream
    time = time[::N] # go from beginning to end of array in steps on N
    time = time[:len(data.streams[GCAMP].data)]


    # Detrending and dFF
    # Full trace dFF according to Lerner et al. 2015
    # http://dx.doi.org/10.1016/j.cell.2015.07.014
    # dFF using 405 fit as baseline

    x = np.array(data.streams[ISOS].data)
    y = np.array(data.streams[GCAMP].data)
    bls = np.polyfit(x, y, 1)
    Y_fit_all = np.multiply(bls[0], x) + bls[1]
    Y_dF_all = y - Y_fit_all

    dFF = zscore(np.multiply(100, np.divide(Y_dF_all, Y_fit_all)))
    return dFF,time

In [ ]:
###get example fp tone trace

id = "004114"
date = "20230922"
sess = 1
areas = get_raw_pupil(id,date)

mat_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
Data = loadmat(mat_path)["data"]["response"]
freqs = Data["trial_freq"]
intensity = Data["trial_dB"]

path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}"
for filename in os.listdir(path):
    if os.path.isdir(os.path.join(path, filename)):
        blockpath = (os.path.join(path, filename))
data = tdt.read_block(blockpath)
GCAMP = '_465A' # GCaMP channel
ISOS = '_405A' # Isosbestic channel
TONE = 'PC2_'
stamps = 'PC0_'
time = np.linspace(1,len(data.streams[GCAMP].data), len(data.streams[GCAMP].data))/data.streams[GCAMP].fs
N = 10 # Average every 10 samples into 1 value
F405 = []
F465 = []

for i in range(0, len(data.streams[GCAMP].data), N):
    F465.append(np.mean(data.streams[GCAMP].data[i:i+N-1])) # This is the moving window mean
data.streams[GCAMP].data = F465

for i in range(0, len(data.streams[ISOS].data), N):
    F405.append(np.mean(data.streams[ISOS].data[i:i+N-1]))
data.streams[ISOS].data = F405

#decimate time array to match length of demodulated stream
time = time[::N] # go from beginning to end of array in steps on N
time = time[:len(data.streams[GCAMP].data)]


# Detrending and dFF
# Full trace dFF according to Lerner et al. 2015
# http://dx.doi.org/10.1016/j.cell.2015.07.014
# dFF using 405 fit as baseline

x = np.array(data.streams[ISOS].data)
y = np.array(data.streams[GCAMP].data)
bls = np.polyfit(x, y, 1)
Y_fit_all = np.multiply(bls[0], x) + bls[1]
Y_dF_all = y - Y_fit_all

dFF = zscore(np.multiply(100, np.divide(Y_dF_all, Y_fit_all)))

x = np.arange(0,data.epocs[stamps].onset[-1],0.05)
xp = data.epocs[stamps].onset[:-1]
fp = areas[:len(data.epocs[stamps].onset)]
areas = np.interp(x,xp,fp)

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,figsize = (6.5,2))

ax1.plot(x,areas,color = "dodgerblue",linewidth = 1)
ax2.plot(time,dFF,color = "green",linewidth = 1)
x_start = 700
x_end = 900
x_space = 5
ylim = [-4,4]


for i,freq in enumerate(freqs):
    if (data.epocs[TONE].onset[i] > x_start) and (data.epocs[TONE].onset[i] < x_end):
        xyA = (data.epocs[TONE].onset[i],3.8)
        xyB = (data.epocs[TONE].onset[i],0.3)
        if freq == 7000:
            #ax1.axvline(data.epocs[TONE].onset[i],color = "black",alpha = 0.5,linestyle = "dashed",linewidth = 1)
            #ax2.axvline(data.epocs[TONE].onset[i],color = "black",alpha = 0.5,linestyle = "dashed")
         

            con = ConnectionPatch(xyA=xyA, coordsA=ax1.transData,xyB=xyB, coordsB=ax2.transData,color = "black",alpha = 0.8,linestyle = "dashed",linewidth = 1)
            fig.add_artist(con)


        if freq == 8000:
            #ax1.axvline(data.epocs[TONE].onset[i],color = "blue",alpha = 0.5,linestyle = "dashed",linewidth = 1)
            #ax2.axvline(data.epocs[TONE].onset[i],color = "blue",alpha = 0.5,linestyle = "dashed")
           

            con = ConnectionPatch(xyA=xyA, coordsA=ax1.transData,xyB=xyB, coordsB=ax2.transData,color = "blue",alpha = 0.8,linestyle = "dashed",linewidth = 1)
            fig.add_artist(con)

        if freq == 9000:
            #ax1.axvline(data.epocs[TONE].onset[i],color = "red",alpha = 0.5,linestyle = "dashed",linewidth = 1)
            #ax2.axvline(data.epocs[TONE].onset[i],color = "red",alpha = 0.5,linestyle = "dashed")
     

            con = ConnectionPatch(xyA=xyA, coordsA=ax1.transData,xyB=xyB, coordsB=ax2.transData,color = "red",alpha = 0.8,linestyle = "dashed",linewidth = 1)
            fig.add_artist(con)


for ax in [ax1,ax2]:
    ax.set_xlim(x_start,x_end)
    ax.set_xticks(np.arange(x_start,x_end + x_space,x_space))
    ax.set_ylim(ylim)
ax2.set_ylim(-2,2)

ax1.spines[['bottom']].set_visible(False)
ax1.spines[['left']].set_visible(False)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)

ax2.spines[['bottom']].set_visible(False)
ax2.spines[['left']].set_visible(False)

ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)

ax2.plot([710,720], [0, 0],color = "black",linewidth = 1)

ax1.plot([710,710], [0, 1],color = "dodgerblue",linewidth = 1)
ax2.plot([710,710], [0, 1],color = "green",linewidth = 1)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_example_trace.pdf",format ="pdf",transparent = True)


In [ ]:
data.epocs[TONE].onset[i]

In [ ]:
def get_pupil_tone(id,date,sess = 1,pupil = True,face = True):
    
    mat_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
    Data = loadmat(mat_path)["data"]["response"]
    freqs = Data["trial_freq"]
    intensity = Data["trial_dB"]

    if pupil:
        areas = get_raw_pupil(id,date)

    path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}"
    for filename in os.listdir(path):
        if os.path.isdir(os.path.join(path, filename)):
            blockpath = (os.path.join(path, filename))
    data = tdt.read_block(blockpath)
    GCAMP = '_465A' # GCaMP channel
    ISOS = '_405A' # Isosbestic channel
    TONE = 'PC2_'
    stamps = 'PC0_'
    time = np.linspace(1,len(data.streams[GCAMP].data), len(data.streams[GCAMP].data))/data.streams[GCAMP].fs
    N = 10 # Average every 10 samples into 1 value
    F405 = []
    F465 = []

    for i in range(0, len(data.streams[GCAMP].data), N):
        F465.append(np.mean(data.streams[GCAMP].data[i:i+N-1])) # This is the moving window mean
    data.streams[GCAMP].data = F465

    for i in range(0, len(data.streams[ISOS].data), N):
        F405.append(np.mean(data.streams[ISOS].data[i:i+N-1]))
    data.streams[ISOS].data = F405

    #decimate time array to match length of demodulated stream
    time = time[::N] # go from beginning to end of array in steps on N
    time = time[:len(data.streams[GCAMP].data)]


    # Detrending and dFF
    # Full trace dFF according to Lerner et al. 2015
    # http://dx.doi.org/10.1016/j.cell.2015.07.014
    # dFF using 405 fit as baseline

    x = np.array(data.streams[ISOS].data)
    y = np.array(data.streams[GCAMP].data)
    bls = np.polyfit(x, y, 1)
    Y_fit_all = np.multiply(bls[0], x) + bls[1]
    Y_dF_all = y - Y_fit_all

    dFF = zscore(np.multiply(100, np.divide(Y_dF_all, Y_fit_all)))
    std_dFF = np.std(dFF)

    if len(data.epocs[TONE].onset) > 100:
        data.epocs[TONE].onset = data.epocs[TONE].onset[:100]


           
    trial_matrix,peri_times = trial_align(data.epocs[TONE].onset,time,dFF,fps = data.streams[GCAMP].fs/N,pre = 10, post = 10)
    x = np.arange(0,data.epocs[stamps].onset[-1],0.05)
    xp = data.epocs[stamps].onset[:-1]
    fp = areas[:len(data.epocs[stamps].onset)]
    areas = np.interp(x,xp,fp)

    pupil_matrix,pupil_peri = trial_align(data.epocs[TONE].onset,x,areas,fps =20 ,pre = 10, post = 10)


    df_list = []
    fps = data.streams[GCAMP].fs/N

    deltas = []
    good_trials = []
    for j,trial in enumerate(pupil_matrix):
        pre_tone = np.nanmean(trial[190:200])
        post_tone = np.nanmean(trial[200:270])
        deltas.append(post_tone-pre_tone)
        if post_tone-pre_tone > 0:
            good_trials.append(j)
            

    
    pre = 10
    for j,trial in enumerate(trial_matrix):
        if j in good_trials:
            baseline = np.mean(trial[int((pre-2)*fps):int((pre)*fps)])
            trial = trial - baseline
            for i,time in enumerate(peri_times):
                new_dict = {"Time from Peak (s)" : time, "trial" : j, "value" : trial[i], "type" : "dFF", "ID" : id, "Date": date,"intensity" : intensity[j],"frequency":freqs[j],"delta":deltas[j]}
                df_list.append(new_dict)    
    

    for j,trial in enumerate(pupil_matrix):
        if j in good_trials:
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri):
                new_dict = {"Time from Peak (s)" : time, "trial" : j, "value" : trial[i], "type" : "Pupil", "ID" : id, "Date": date,"intensity" : intensity[j],"frequency":freqs[j],'delta':deltas[j]}
                df_list.append(new_dict)

   
    aligned_df = pd.DataFrame.from_dict(df_list)
    labels = ["1","2","3","4"]
    aligned_df["bin"] = pd.qcut(aligned_df["delta"],4,labels = labels)

    return aligned_df


In [ ]:
###average aligned trace
id_date = {
"004113":["20230922"],
"004114":["20230922"],
"004115":["20230922"],
"004116":["20230922"],
"004117":["20230922"],
"004118":["20230922"],
}

dfs = []
for id,dates in id_date.items():
    for date in dates:
        dfs.append(get_pupil_tone(id,date))
df = pd.concat(dfs)
tone_df = df.reset_index()

In [ ]:
###average aligned trace
plt.figure(figsize = (1.4,1))

ax = sns.lineplot(data = tone_df.groupby(["ID","Time from Peak (s)","type","frequency","bin"]).mean().reset_index(), x = "Time from Peak (s)", y = "value",errorbar = "se",palette = ["dodgerblue","green"],linewidth = 1,hue = "type",legend = None)
format_ax(ax,[-1,4],[-0.2,1],1,0.2)
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
ax.set_xlabel("Time from Tone (s)")
ax.set_ylabel("Z-Score")
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/fp_toneall.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###average aligned trace
plt.figure(figsize = (1,1))

ax = sns.lineplot(data = tone_df.query("bin == '1'").groupby(["ID","Time from Peak (s)","type","frequency","bin"]).mean().reset_index(), x = "Time from Peak (s)", y = "value",errorbar = "se",palette = ["dodgerblue","green"],linewidth = 1,hue = "type",legend = None)
format_ax(ax,[-1,4],[-0.4,2 ],1,0.4)
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
ax.set_xlabel("Time from Tone (s)")
ax.set_ylabel("Z-Score")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/fp_toneq1.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###average aligned trace
plt.figure(figsize = (1,1))

ax = sns.lineplot(data = tone_df.query("bin == '4'").groupby(["ID","Time from Peak (s)","type","frequency","bin"]).mean().reset_index(), x = "Time from Peak (s)", y = "value",errorbar = "se",palette = ["dodgerblue","green"],linewidth = 1,hue = "type",legend = None)
format_ax(ax,[-1,4],[-0.4,2],1,0.4)
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
ax.set_xlabel("Time from Tone (s)")
ax.set_ylabel("Z-Score")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/fp_toneq4.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
def get_tone_metrics(id,date,sess = 1,pupil = True,face = True):
    
    mat_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
    Data = loadmat(mat_path)["data"]["response"]
    freqs = Data["trial_freq"]
    intensity = Data["trial_dB"]

    if pupil:
        areas = get_raw_pupil(id,date)

    path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}"
    for filename in os.listdir(path):
        if os.path.isdir(os.path.join(path, filename)):
            blockpath = (os.path.join(path, filename))
    data = tdt.read_block(blockpath)
    GCAMP = '_465A' # GCaMP channel
    ISOS = '_405A' # Isosbestic channel
    TONE = 'PC2_'
    stamps = 'PC0_'
    time = np.linspace(1,len(data.streams[GCAMP].data), len(data.streams[GCAMP].data))/data.streams[GCAMP].fs
    N = 10 # Average every 10 samples into 1 value
    F405 = []
    F465 = []

    for i in range(0, len(data.streams[GCAMP].data), N):
        F465.append(np.mean(data.streams[GCAMP].data[i:i+N-1])) # This is the moving window mean
    data.streams[GCAMP].data = F465

    for i in range(0, len(data.streams[ISOS].data), N):
        F405.append(np.mean(data.streams[ISOS].data[i:i+N-1]))
    data.streams[ISOS].data = F405

    #decimate time array to match length of demodulated stream
    time = time[::N] # go from beginning to end of array in steps on N
    time = time[:len(data.streams[GCAMP].data)]


    # Detrending and dFF
    # Full trace dFF according to Lerner et al. 2015
    # http://dx.doi.org/10.1016/j.cell.2015.07.014
    # dFF using 405 fit as baseline

    x = np.array(data.streams[ISOS].data)
    y = np.array(data.streams[GCAMP].data)
    bls = np.polyfit(x, y, 1)
    Y_fit_all = np.multiply(bls[0], x) + bls[1]
    Y_dF_all = y - Y_fit_all

    dFF = zscore(np.multiply(100, np.divide(Y_dF_all, Y_fit_all)))
    std_dFF = np.std(dFF)

    if len(data.epocs[TONE].onset) > 100:
        data.epocs[TONE].onset = data.epocs[TONE].onset[:100]


           
    trial_matrix,peri_times = trial_align(data.epocs[TONE].onset,time,dFF,fps = data.streams[GCAMP].fs/N,pre = 10, post = 10)
    x = np.arange(0,data.epocs[stamps].onset[-1],0.05)
    xp = data.epocs[stamps].onset[:-1]
    fp = areas[:len(data.epocs[stamps].onset)]
    areas = np.interp(x,xp,fp)

    pupil_matrix,pupil_peri = trial_align(data.epocs[TONE].onset,x,areas,fps =20 ,pre = 10, post = 10)


    df_list = []
    fps = data.streams[GCAMP].fs/N

    deltas = []
    good_trials = []
    for j,trial in enumerate(pupil_matrix):
        pre_tone = np.nanmean(trial[160:200])
        post_tone = np.nanmean(trial[200:270])
        deltas.append(post_tone-pre_tone)

        if post_tone-pre_tone>0:
            good_trials.append(j)
            

    
    pre = 10
    for j,trial in enumerate(trial_matrix):
        if j in good_trials:
            pre_tone = np.mean(trial[int((pre-2)*fps):int((pre)*fps)])
            post_tone = np.mean(trial[int((pre)*fps):int((pre+3.5)*fps)])
            new_dict = {"trial" : j, "dFF" : post_tone - pre_tone, "Pupil" : deltas[j], "ID" : id, "Date": date,"intensity" : intensity[j],"frequency":str(freqs[j]/1000)}
            df_list.append(new_dict)    

    aligned_df = pd.DataFrame.from_dict(df_list)
    labels = ["1","2","3","4"]
    aligned_df["bin"] = pd.qcut(aligned_df["Pupil"],4,labels = labels)

    return aligned_df


In [ ]:
###tone delta bar

id_date = {
"004113":["20230922"],
"004114":["20230922"],
"004115":["20230922"],
"004116":["20230922"],
"004117":["20230922"],
"004118":["20230922"],
}


dfs = []
for id,dates in id_date.items():
    for date in dates:
        dfs.append(get_tone_metrics(id,date))
df = pd.concat(dfs)
bar_df = df.reset_index()

In [ ]:
plt.figure(figsize = (1,1))
g = sns.barplot(data = bar_df.groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "dFF",errorbar = "se",fill = None,errwidth= 1,edgecolor = "green")
sns.lineplot(data = bar_df.groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "dFF",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Tone Freq.(kHz.)")
g.set_ylabel("∆F/F")


g.set_yticks(np.arange(0,1,0.25))
g.set(ylim=(0,0.75))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_delta_freqbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (1,1))
g = sns.barplot(data = bar_df.groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Pupil",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.lineplot(data = bar_df.groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Pupil",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("Tone Freq.(kHz.)")
g.set_ylabel("∆ Pupil Size\n(z-scr)")


g.set_yticks(np.arange(0,2,0.5))
g.set(ylim=(0,1.5))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_delta_pupilbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'frequency']).mean().reset_index().query("frequency == '7.0'")["Pupil"]
q2 = bar_df.groupby(["ID",'frequency']).mean().reset_index().query("frequency == '8.0'")["Pupil"]
q3 = bar_df.groupby(["ID",'frequency']).mean().reset_index().query("frequency == '9.0'")["Pupil"]
kruskal(q1,q2,q3)

In [ ]:
###dff responses over time
def label_trial(row):
    if row['trial']<33:
      return 'Early'
    if row['trial']>67:
      return 'Late'
    else:
      return "Middle"

bar_df['Time'] = bar_df.apply(label_trial, axis=1)

In [ ]:
bar_df

In [ ]:
plt.figure(figsize = (1,1))
g = sns.barplot(data = bar_df.groupby(["ID",'Time']).mean().reset_index(), x = "Time", y = "dFF",errorbar = "se",fill = None,errwidth= 1,edgecolor = "green",order = ["Early","Middle","Late"])
sns.lineplot(data = bar_df.groupby(["ID",'Time']).mean().reset_index(), x = "Time", y = "dFF",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[3:]:
    y1,y2,y3 = line.get_ydata()
    line.set_ydata([y1,y3,y2])
g.set_xlabel("Time in Session")
g.set_ylabel("∆F/F")


g.set_yticks(np.arange(0,1,0.25))
g.set(ylim=(0,0.75))
g.tick_params(axis='x', labelrotation=45)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_dff_habituation.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (1,1))
g = sns.barplot(data = bar_df.groupby(["ID",'Time']).mean().reset_index(), x = "Time", y = "Pupil",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue",order = ["Early","Middle","Late"])
sns.lineplot(data = bar_df.groupby(["ID",'Time']).mean().reset_index(), x = "Time", y = "Pupil",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[3:]:
    y1,y2,y3 = line.get_ydata()
    line.set_ydata([y1,y3,y2])
g.set_xlabel("Time in Session")
g.set_ylabel("∆ Pupil Size\n(z-scr)")


g.set_yticks(np.arange(0,2,0.5))
g.set(ylim=(0,1.5))
g.tick_params(axis='x', labelrotation=45)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/S4_pupil_habituation.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (1,1))
g = sns.barplot(data = bar_df.groupby(["ID",'bin']).mean().reset_index(), x = "bin", y = "dFF",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.lineplot(data = bar_df.groupby(["ID",'bin']).mean().reset_index(), x = "bin", y = "dFF",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xlabel("∆ Pupil Size\n Quartile")
g.set_ylabel("∆ ∆F/F")

g.set_yticks(np.arange(0,1.25,0.25))
g.set(ylim=(0,1))
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'bin']).mean().reset_index().query("bin == '1'")["dFF"]
q2 = bar_df.groupby(["ID",'bin']).mean().reset_index().query("bin == '2'")["dFF"]
q3 = bar_df.groupby(["ID",'bin']).mean().reset_index().query("bin == '3'")["dFF"]
q4 = bar_df.groupby(["ID",'bin']).mean().reset_index().query("bin == '4'")["dFF"]


kruskal(q1,q2,q3,q4)

In [ ]:
###habitution stats
q1 = bar_df.groupby(["ID",'Time']).mean().reset_index().query("Time == 'Early'")["Pupil"]
q2 = bar_df.groupby(["ID",'Time']).mean().reset_index().query("Time == 'Middle'")["Pupil"]
q3 = bar_df.groupby(["ID",'Time']).mean().reset_index().query("Time == 'Late'")["Pupil"]


kruskal(q1,q2,q3)

In [ ]:
###duration histogram
plt.figure(figsize = (1,1))
deltas = bar_df["Pupil"]
deltas = np.clip(deltas,a_max = 2,a_min = 0)
g = sns.histplot(data = deltas,bins =[0,0.25,0.5,0.75,1,1.25,1.5,1.75,2],stat = "percent",fill =False,color = "dodgerblue",linewidth = 1)
g.set_xlabel("∆ Pupil Size\n(z-scr)")
g.set_xticks(np.arange(0,3,0.25))
g.set(xlim=(0,2))
g.set_yticks(np.arange(0,80,20))
g.set(ylim=(0,40))
labels = [item.get_text() for item in plt.gca().get_xticklabels()]
labels[-1] = '2+'
plt.gca().set_xticklabels(labels)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_pupil_hist.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###tone opto not baseline corrected
id_date = {   
            "004072" : ["20230801","20230803","20230804"],
            "004075" : ["20230801","20230803","20230804"],
            "004073" : ["20230803","20230804"],
            "004109" : ["20230831","20230905"],
            "004110" : ["20230905"],
            "004111" : ["20230831","20230905"],
            "004112" : ["20230831","20230905"],   
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        tone_times = mat_data["response"]["tone_time"]

        data = mat_data["response"]["sessdata"]
        trial_types = mat_data["response"]["trial_type"]
        trial_freqs = mat_data["response"]["trial_freq"]
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        pupil_snips,pupil_peri_times = trial_align(tone_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            # baseline = np.nanmean(trial[160:200])
            # trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Tone (s)" : time, "trial" : j, "value" : trial[i],"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"condition":trial_types[j],"freq":trial_freqs[j]}
                df_list.append(new_dict)
vgat_df = pd.DataFrame.from_dict(df_list)

In [ ]:
ax = sns.relplot(data = vgat_df, x = "Time from Tone (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,hue = "condition",legend = None,palette = ["black","dodgerblue"],kind = "line",col = "ID")


In [ ]:
plt.figure(figsize = (1.1,1))

ax = sns.lineplot(data = vgat_df.groupby(["ID","Time from Tone (s)","type","condition"]).mean().reset_index(), x = "Time from Tone (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,hue = "condition",legend = None,palette = ["black","dodgerblue"])
ax.axvspan(-5,5,alpha = 0.2)
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,[-6,8],[-1,1],2,0.5)
ax.set_xlabel("Time from Tone (s)")
ax.set_ylabel("Pupil Size (z-scr)")
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_opto_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###tone opto baseline corrected
id_date = {   
            "004072" : ["20230801","20230803","20230804"],
            "004075" : ["20230801","20230803","20230804"],
            "004073" : ["20230803","20230804"],
            "004109" : ["20230831","20230905"],
            "004110" : ["20230905"],
            "004111" : ["20230831","20230905"],
            "004112" : ["20230831","20230905"],   
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        tone_times = mat_data["response"]["tone_time"]

        data = mat_data["response"]["sessdata"]
        trial_types = mat_data["response"]["trial_type"]
        trial_freqs = mat_data["response"]["trial_freq"]
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        pupil_snips,pupil_peri_times = trial_align(tone_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            baseline = np.nanmean(trial[190:210])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Tone (s)" : time, "trial" : j, "value" : trial[i],"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"condition":trial_types[j],"freq":trial_freqs[j]}
                df_list.append(new_dict)
corrected_df = pd.DataFrame.from_dict(df_list)

In [ ]:
ax = sns.relplot(data = corrected_df, x = "Time from Tone (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,hue = "condition",legend = None,palette = ["black","dodgerblue"],col = "ID",kind = "line")


In [ ]:
plt.figure(figsize = (1.1,1))

ax = sns.lineplot(data = corrected_df.groupby(["ID","Time from Tone (s)","type","condition"]).mean().reset_index(), x = "Time from Tone (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,hue = "condition",legend = None,palette = ["black","dodgerblue"])
ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)
format_ax(ax,[-1,4],[-0.25,0.5],1,0.25)
ax.set_xlabel("Time from tone (s)")
ax.set_ylabel("Pupil size (z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_corrected_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###tone opto not baseline corrected
id_date = {   
            "004072" : ["20230801","20230803","20230804"],
            "004075" : ["20230801","20230803","20230804"],
            "004073" : ["20230803","20230804"],
            "004109" : ["20230831","20230905"],
            "004110" : ["20230905"],
            "004111" : ["20230831","20230905"],
            "004112" : ["20230831","20230905"],   
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        tone_times = mat_data["response"]["tone_time"]

        data = mat_data["response"]["sessdata"]
        trial_types = mat_data["response"]["trial_type"]
        trial_freqs = mat_data["response"]["trial_freq"]
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        pupil_snips,pupil_peri_times = trial_align(tone_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[60:100])
            post_laser = np.nanmean(trial[100:300])
        
            new_dict = {"trial" : j, "Delta" : post_laser - pre_laser,"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"condition":trial_types[j],"freq":trial_freqs[j]}
            df_list.append(new_dict)
vgat_bar_df = pd.DataFrame.from_dict(df_list)

In [ ]:
plt.figure(figsize = (0.75,1))
g = sns.barplot(data = vgat_bar_df.groupby(["ID",'condition']).mean().reset_index(), x = "condition", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.lineplot(data = vgat_bar_df.groupby(["ID",'condition']).mean().reset_index(), x = "condition", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_ylabel("∆ Pupil Size")

g.set_yticks(np.arange(-1,0.75,0.25))
g.set(ylim=(-0.75,0.5))
g.set_xticklabels(["Non-Laser","Laser"])
g.xaxis.label.set_visible(False)
g.tick_params(axis='x', labelrotation=45)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = vgat_bar_df.groupby(["ID",'condition']).mean().reset_index()
group1 = stat_df[stat_df['condition']==1]
group2 = stat_df[stat_df['condition']==0]
wilcoxon(group1["Delta"], group2["Delta"],nan_policy= "omit")

In [ ]:
###tone opto not baseline corrected
id_date = {   
            "004072" : ["20230801","20230803","20230804"],
            "004075" : ["20230801","20230803","20230804"],
            "004073" : ["20230803","20230804"],
            "004109" : ["20230831","20230905"],
            "004110" : ["20230905"],
            "004111" : ["20230831","20230905"],
            "004112" : ["20230831","20230905"],   
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        tone_times = mat_data["response"]["tone_time"]

        data = mat_data["response"]["sessdata"]
        trial_types = mat_data["response"]["trial_type"]
        trial_freqs = mat_data["response"]["trial_freq"]
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        pupil_snips,pupil_peri_times = trial_align(tone_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[180:200])
            post_laser = np.nanmean(trial[200:260])
        
            new_dict = {"trial" : j, "Delta" : post_laser - pre_laser,"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"condition":trial_types[j],"freq":trial_freqs[j]}
            df_list.append(new_dict)
corrected_bar_df = pd.DataFrame.from_dict(df_list)

In [ ]:
plt.figure(figsize = (0.75,1))
g = sns.barplot(data = corrected_bar_df.groupby(["ID",'condition']).mean().reset_index(), x = "condition", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.lineplot(data = corrected_bar_df.groupby(["ID",'condition']).mean().reset_index(), x = "condition", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_ylabel("∆ Pupil Size")

g.set_yticks(np.arange(-1,0.75,0.25))
g.set(ylim=(-0.25,0.5))
g.set_xticklabels(["Non-Laser","Laser"])
g.xaxis.label.set_visible(False)
g.tick_params(axis='x', labelrotation=45)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/corrected_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = corrected_bar_df.groupby(["ID",'condition']).mean().reset_index()
group1 = stat_df[stat_df['condition']==1]
group2 = stat_df[stat_df['condition']==0]
wilcoxon(group1["Delta"], group2["Delta"],nan_policy= "omit")

In [ ]:
###habituation analysis

def label_trial(row):
    if row['trial']<30:
      return 'Early'
    if row['trial']>60:
      return 'Late'
    return 'Middle'

corrected_bar_df['Time'] = corrected_bar_df.apply(label_trial, axis=1)


plt.figure(figsize = (1,1))
g = sns.barplot(data = corrected_bar_df.query("condition == 0").groupby(["ID",'Time']).mean().reset_index(), x = "Time", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue",order = ["Early","Middle","Late"])
sns.lineplot(data = corrected_bar_df.query("condition == 0").groupby(["ID",'Time']).mean().reset_index(), x = "Time", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[3:]:
    y1,y2,y3 = line.get_ydata()
    line.set_ydata([y1,y3,y2])

g.set_ylabel("∆ Pupil Size")

g.set_yticks(np.arange(0,1,0.2))
g.set(ylim=(0,0.8))
g.xaxis.label.set_visible(False)
g.tick_params(axis='x', labelrotation=45)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/tone_habituation_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = corrected_bar_df.query("condition == 0").groupby(["ID",'Time']).mean().reset_index()

q1 = stat_df.query("Time == 'Early'")["Delta"]
q2 = stat_df.query("Time == 'Middle'")["Delta"]
q3 = stat_df.query("Time == 'Late'")["Delta"]
kruskal(q1,q2,q3)

In [ ]:
#plt.figure(figsize = (0.75,1))
g = sns.barplot(data = corrected_bar_df.groupby(["ID",'condition',"freq"]).mean().reset_index(), x = "freq", y = "Delta",errorbar = "se",fill = None,errwidth= 1,hue = "condition")
#sns.lineplot(data = corrected_bar_df.groupby(["ID",'condition',"freq"]).mean().reset_index(), x = "condition", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_ylabel("∆ Pupil Size")

g.set_yticks(np.arange(-1,0.75,0.25))
g.set(ylim=(-0.25,0.5))
g.set_xticklabels(["Non-Laser","Laser"])
g.xaxis.label.set_visible(False)
g.tick_params(axis='x', labelrotation=45)
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/corrected_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")
